<a href="https://colab.research.google.com/github/Nicordaro/Project_MLDL/blob/master/Project_MLDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Politecnico di Torino**

**01TXFSM - Machine learning and Deep learning**

**Incremental Learning in Image Classification**

**Cordaro Nicolò - s272145**

**Di Nepi Marco - sMATRICOLA**

**Falletta Alberto - s277971**


In [0]:
# !pip3 install 'torch==1.4.0'
# !pip3 install 'torchvision==0.5.0'
# !pip3 install 'Pillow-SIMD'
# !pip3 install 'tqdm'

**Imports**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn


import torchvision
from torchvision import transforms
from torchvision.models import resnet18

from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from google.colab import drive

**Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

# Init at 10 because first train is on 10 classes
NUM_CLASSES = 10

# Used for the pseudorandom shuffle of the split
SEED = 42

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Pre-processing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # https://github.com/pytorch/examples/blob/master/imagenet/main.py
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))                                 
])

**Prepare Dataset**

CIFAR100 has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another.

Each of the downloaded files is a Python "pickled" object produced with cPickle.

In [12]:
# Clone github repository with data
if os.path.isdir('./Project_MLDL'):
  !rm -rf Project_MLDL
if not os.path.isdir('./CIFAR100_tError'):
  !git clone https://github.com/Nicordaro/Project_MLDL


Cloning into 'Project_MLDL'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [0]:
from Project_MLDL.CIFAR100_tError import CIFAR100_tError
import numpy as np
import random

DATA_DIR = './CIFAR100'

lbls = [i for i in range(0,100)]
random.seed(SEED)
random.shuffle(lbls)

added_labels=[]
new_labels=[]

def make_data_labels(lbls):
  new_labels=[]
  for el in lbls[:10]:
    added_labels.append(el)
    new_labels.append(el)
  lbls = lbls[10:]

  return added_labels, lbls, new_labels

def increment_dataset(new_labels):
  CIFAR100_tError.increment(new_labels)

data -- a 50000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

labels -- a list of 50000 numbers in the range 0-99. The number at index i indicates the label of the ith image in the array data.

**Create dataset**

In [26]:
if not os.path.isdir('./CIFAR100'):
  train_dataset = CIFAR100_tError(DATA_DIR, train=True, transform=None, target_transform=None, download=True)
  test_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=None, target_transform=None, download=True)

TypeError: ignored

**Dataloaders**

In [0]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

**Prepare Network**

In [0]:
net = resnet18(pretrained=False)
net.fc = nn.Linear(512, NUM_CLASSES)

**Prepare Training**

In [0]:
# Loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Parameters to optimize:
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
parameters_to_optimize = net.parameters()

# Optimizers
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Scheduler
# A scheduler dynamically changes learning rate
# The most common scheduler is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)